In [8]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import spotipy.util as util
import warnings

In [2]:
with open('clientspotify.txt', 'r') as file:
    cid = file.read()
with open('secretspotify.txt', 'r') as file:
    secret = file.read()
client_credentials_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [3]:
username = 'maddiekay13'
scope_top_tracks = 'user-top-read'
scope_saved_tracks = 'user-library-read'

sp2 = spotipy.Spotify(auth_manager = SpotifyOAuth(scope = scope_saved_tracks, client_id = cid, client_secret = secret, redirect_uri = 'http://localhost:8080', username = username))

In [4]:
offset = 0
limit = 50

names = []
ids = []
release_dates = []
popularity = []
uris = []

my_tracks = sp2.current_user_saved_tracks(limit = limit, offset = offset)

while(len(my_tracks['items']) > 0):
    for i in range(50):
        if len(my_tracks['items']) >= i+1:
            try:
                current_track = my_tracks['items'][i]['track']
                
                uris.append(current_track['uri'])
                names.append(current_track['name'])
                ids.append(current_track['id'])
                release_dates.append(current_track['album']['release_date'])
                popularity.append(current_track['popularity'])
            except:
                print(i)
                
    offset = offset + 50
    my_tracks = sp2.current_user_saved_tracks(limit = limit, offset = offset)

In [5]:
len(uris)

697

In [6]:
track_features = []
for uri in uris:
    track_features.append(sp2.audio_features(uri))

In [9]:
audio_features = pd.DataFrame(track_features[0])

with warnings.catch_warnings(record=True):
    for i in range(1, len(track_features)):
        audio_features = audio_features.append(pd.DataFrame(track_features[i]))
        
    
audio_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.672,0.072,2,-13.406,1,0.0413,0.9600,0.000689,0.1200,0.369,71.899,audio_features,4sKZlRkHsoovZB9M3XbP1u,spotify:track:4sKZlRkHsoovZB9M3XbP1u,https://api.spotify.com/v1/tracks/4sKZlRkHsoov...,https://api.spotify.com/v1/audio-analysis/4sKZ...,190187,4
0,0.692,0.758,3,-3.453,0,0.0522,0.1580,0.000000,0.0774,0.701,94.058,audio_features,3UEOT0hD60yGB1OMWJxLQ8,spotify:track:3UEOT0hD60yGB1OMWJxLQ8,https://api.spotify.com/v1/tracks/3UEOT0hD60yG...,https://api.spotify.com/v1/audio-analysis/3UEO...,146654,4
0,0.653,0.575,5,-8.950,0,0.0594,0.5370,0.000079,0.1210,0.409,119.888,audio_features,5FGDtm3MXJe8LikBkOJnOQ,spotify:track:5FGDtm3MXJe8LikBkOJnOQ,https://api.spotify.com/v1/tracks/5FGDtm3MXJe8...,https://api.spotify.com/v1/audio-analysis/5FGD...,245629,4
0,0.633,0.761,11,-7.097,1,0.0340,0.0502,0.001140,0.6380,0.801,96.061,audio_features,21lfQ6cXJF15ubyPLB6qHz,spotify:track:21lfQ6cXJF15ubyPLB6qHz,https://api.spotify.com/v1/tracks/21lfQ6cXJF15...,https://api.spotify.com/v1/audio-analysis/21lf...,248438,4
0,0.417,0.487,1,-6.128,0,0.0367,0.2720,0.000000,0.2350,0.327,112.338,audio_features,17YdABkVGTOe5f3JQBRAAy,spotify:track:17YdABkVGTOe5f3JQBRAAy,https://api.spotify.com/v1/tracks/17YdABkVGTOe...,https://api.spotify.com/v1/audio-analysis/17Yd...,239499,4


In [10]:
track_info = pd.DataFrame(list(zip(names, uris, popularity, release_dates)), columns = ['name', 'uri', 'popularity', 'release_date'])
track_info.head()

,name,uri,popularity,release_date
0,Recency Bias,spotify:track:4sKZlRkHsoovZB9M3XbP1u,35,2023-03-10
1,people pleaser,spotify:track:3UEOT0hD60yGB1OMWJxLQ8,73,2022-10-07
2,I know it won't work,spotify:track:5FGDtm3MXJe8LikBkOJnOQ,73,2023-02-24
3,Vanilla,spotify:track:21lfQ6cXJF15ubyPLB6qHz,51,2017-03-31
4,If You Want Me Tonight,spotify:track:17YdABkVGTOe5f3JQBRAAy,49,2023-03-01


In [11]:
my_tracks = pd.merge(track_info, audio_features, on = 'uri').drop(['type', 'track_href', 'analysis_url'],axis=1)
my_tracks.head()

,name,uri,popularity,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,Recency Bias,spotify:track:4sKZlRkHsoovZB9M3XbP1u,35,2023-03-10,0.672,0.072,2,-13.406,1,0.0413,0.9600,0.000689,0.1200,0.369,71.899,4sKZlRkHsoovZB9M3XbP1u,190187,4
1,people pleaser,spotify:track:3UEOT0hD60yGB1OMWJxLQ8,73,2022-10-07,0.692,0.758,3,-3.453,0,0.0522,0.1580,0.000000,0.0774,0.701,94.058,3UEOT0hD60yGB1OMWJxLQ8,146654,4
2,I know it won't work,spotify:track:5FGDtm3MXJe8LikBkOJnOQ,73,2023-02-24,0.653,0.575,5,-8.950,0,0.0594,0.5370,0.000079,0.1210,0.409,119.888,5FGDtm3MXJe8LikBkOJnOQ,245629,4
3,Vanilla,spotify:track:21lfQ6cXJF15ubyPLB6qHz,51,2017-03-31,0.633,0.761,11,-7.097,1,0.0340,0.0502,0.001140,0.6380,0.801,96.061,21lfQ6cXJF15ubyPLB6qHz,248438,4
4,If You Want Me Tonight,spotify:track:17YdABkVGTOe5f3JQBRAAy,49,2023-03-01,0.417,0.487,1,-6.128,0,0.0367,0.2720,0.000000,0.2350,0.327,112.338,17YdABkVGTOe5f3JQBRAAy,239499,4


In [12]:
my_tracks.to_csv('my_saved_tracks.csv')